# logging_config 테스트

In [12]:
import sys
!{sys.executable} -m pip install pytz
!{sys.executable} -m pip install pyyaml
!{sys.executable} -m pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 25.3 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 77.3 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [numpy]  WARNING: The scripts f2py and numpy-config are installed in '/home/spai0433/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]2m2/3 [pandas]


In [2]:
import sys
import logging
from pathlib import Path
from datetime import datetime
import shutil
import pytz
import yaml

# 프로젝트 루트를 sys.path에 추가
project_root = Path(__file__).resolve().parent.parent if '__file__' in globals() else Path.cwd().parent
sys.path.insert(0, str(project_root))

from src.utils.logging_config import (
    setup_logger,
    get_logger,
    reset_logger,
    reset_all_loggers,
    set_level,
    get_log_file_path,
    ShortLevelFormatter
)

## 테스트 준비

In [3]:
# 테스트 결과 저장용
test_results = []
test_log_dir = project_root / "logs" / "test"

def add_test_result(test_name: str, status: str, details: str = ""):
    """테스트 결과 추가"""
    test_results.append({
        "테스트명": test_name,
        "상태": status,
        "상세내용": details
    })
    print(f"[{status}] {test_name}: {details}")

# 기존 로거 모두 초기화 (파일 핸들러 닫기)
reset_all_loggers()

# 테스트 로그 디렉토리 생성
test_log_dir.mkdir(parents=True, exist_ok=True)

print(f"테스트 준비 완료")
print(f"프로젝트 루트: {project_root}")
print(f"테스트 로그 디렉토리: {test_log_dir}")

테스트 준비 완료
프로젝트 루트: /home/spai0433/DATA/Dev/Work/codeit_ai_g2b_search
테스트 로그 디렉토리: /home/spai0433/DATA/Dev/Work/codeit_ai_g2b_search/logs/test


## Test 1: 로거 생성 및 기본 설정

In [4]:
try:
    reset_all_loggers()
    logger = setup_logger("test_logger_1", log_dir=str(test_log_dir))
    
    # 로거가 생성되었는지 확인
    assert logger is not None, "로거가 생성되지 않음"
    assert logger.name == "test_logger_1", "로거 이름이 일치하지 않음"
    assert len(logger.handlers) > 0, "핸들러가 추가되지 않음"
    
    # 기본 로그 작성 테스트
    logger.info("테스트 로그 메시지")
    logger.debug("디버그 메시지")
    logger.warning("경고 메시지")
    logger.error("에러 메시지")
    
    add_test_result("로거 생성 및 기본 설정", "PASS", "로거 생성 및 로그 작성 성공")
except Exception as e:
    add_test_result("로거 생성 및 기본 설정", "FAIL", str(e))

⚠ 설정 파일을 찾을 수 없습니다: config/config.json
기본 설정을 사용합니다.
✓ 설정 검증 통과
2025-11-10 17:52:57 [I] test_logger_1 - 테스트 로그 메시지
2025-11-10 17:52:57 [W] test_logger_1 - 경고 메시지
2025-11-10 17:52:57 [E] test_logger_1 - 에러 메시지
[PASS] 로거 생성 및 기본 설정: 로거 생성 및 로그 작성 성공


## Test 2: 파일 핸들러 및 로그 파일 생성

In [5]:
try:
    reset_all_loggers()
    logger = setup_logger("test_logger_2", enable_file=True, log_dir=str(test_log_dir))
    
    # 로그 파일 경로 확인
    log_file_path = get_log_file_path("test_logger_2")
    assert log_file_path is not None, "로그 파일 경로를 찾을 수 없음"
    
    # 로그 작성
    logger.info("파일 핸들러 테스트 메시지")
    logger.error("에러 메시지 테스트")
    
    # 파일이 생성되었는지 확인
    assert log_file_path.exists(), "로그 파일이 생성되지 않음"
    
    # 파일 내용 확인
    log_content = log_file_path.read_text(encoding='utf-8')
    assert "파일 핸들러 테스트 메시지" in log_content, "로그 내용이 파일에 기록되지 않음"
    
    add_test_result("파일 핸들러 및 로그 파일 생성", "PASS", f"로그 파일: {log_file_path}")
except Exception as e:
    add_test_result("파일 핸들러 및 로그 파일 생성", "FAIL", str(e))

2025-11-10 17:52:57 [I] test_logger_2 - 파일 핸들러 테스트 메시지
2025-11-10 17:52:57 [E] test_logger_2 - 에러 메시지 테스트
[PASS] 파일 핸들러 및 로그 파일 생성: 로그 파일: /home/spai0433/DATA/Dev/Work/codeit_ai_g2b_search/logs/test/rag_system.log


## Test 3: 로거 캐싱 및 재사용

In [6]:
try:
    reset_all_loggers()
    logger1 = setup_logger("test_logger_cache", log_dir=str(test_log_dir))
    logger2 = get_logger("test_logger_cache")
    logger3 = setup_logger("test_logger_cache", log_dir=str(test_log_dir))
    
    # 모두 동일한 인스턴스여야 함
    assert logger1 is logger2, "get_logger가 캐시된 로거를 반환하지 않음"
    assert logger1 is logger3, "setup_logger가 기존 로거를 재사용하지 않음"
    
    add_test_result("로거 캐싱 및 재사용", "PASS", "동일 인스턴스 재사용 확인")
except Exception as e:
    add_test_result("로거 캐싱 및 재사용", "FAIL", str(e))

[PASS] 로거 캐싱 및 재사용: 동일 인스턴스 재사용 확인


## Test 4: 로깅 레벨 변경

In [7]:
try:
    reset_all_loggers()
    logger = setup_logger("test_logger_level", level=logging.INFO, log_dir=str(test_log_dir))
    
    # 초기 레벨 확인
    assert logger.level == logging.INFO, "초기 레벨이 INFO가 아님"
    
    # 레벨 변경
    set_level("test_logger_level", logging.DEBUG)
    assert logger.level == logging.DEBUG, "레벨이 DEBUG로 변경되지 않음"
    
    # 핸들러 레벨도 변경되었는지 확인
    for handler in logger.handlers:
        assert handler.level == logging.DEBUG, "핸들러 레벨이 변경되지 않음"
    
    add_test_result("로깅 레벨 변경", "PASS", "INFO → DEBUG 변경 성공")
except Exception as e:
    add_test_result("로깅 레벨 변경", "FAIL", str(e))

[PASS] 로깅 레벨 변경: INFO → DEBUG 변경 성공


## Test 5: ShortLevelFormatter 동작 확인

In [8]:
try:
    reset_all_loggers()
    logger = setup_logger("test_logger_formatter", level=logging.DEBUG, log_dir=str(test_log_dir))
    
    # 로그 작성
    logger.debug("Debug message")
    logger.info("Info message")
    logger.warning("Warning message")
    logger.error("Error message")
    logger.critical("Critical message")
    
    # 로그 파일 내용 확인
    log_file_path = get_log_file_path("test_logger_formatter")
    assert log_file_path is not None, "로그 파일을 찾을 수 없음"
    
    log_content = log_file_path.read_text(encoding='utf-8')
    
    # 축약된 레벨 확인
    assert "[D]" in log_content, "DEBUG 레벨이 D로 축약되지 않음"
    assert "[I]" in log_content, "INFO 레벨이 I로 축약되지 않음"
    assert "[W]" in log_content, "WARNING 레벨이 W로 축약되지 않음"
    assert "[E]" in log_content, "ERROR 레벨이 E로 축약되지 않음"
    assert "[C]" in log_content, "CRITICAL 레벨이 C로 축약되지 않음"
    
    add_test_result("ShortLevelFormatter 동작", "PASS", "D/I/W/E/C 축약 확인")
except Exception as e:
    add_test_result("ShortLevelFormatter 동작", "FAIL", str(e))

2025-11-10 17:52:57 [D] test_logger_formatter - Debug message
2025-11-10 17:52:57 [I] test_logger_formatter - Info message
2025-11-10 17:52:57 [W] test_logger_formatter - Warning message
2025-11-10 17:52:57 [E] test_logger_formatter - Error message
2025-11-10 17:52:57 [C] test_logger_formatter - Critical message
[PASS] ShortLevelFormatter 동작: D/I/W/E/C 축약 확인


## Test 6: 콘솔/파일 핸들러 선택적 활성화

In [9]:
try:
    reset_all_loggers()
    
    # 콘솔만 활성화
    logger_console = setup_logger("test_console_only", enable_console=True, enable_file=False, log_dir=str(test_log_dir))
    assert len(logger_console.handlers) == 1, "콘솔 전용 로거에 핸들러가 1개가 아님"
    assert isinstance(logger_console.handlers[0], logging.StreamHandler), "콘솔 핸들러가 아님"
    
    # 파일만 활성화
    logger_file = setup_logger("test_file_only", enable_console=False, enable_file=True, log_dir=str(test_log_dir))
    assert len(logger_file.handlers) == 1, "파일 전용 로거에 핸들러가 1개가 아님"
    assert isinstance(logger_file.handlers[0], logging.handlers.RotatingFileHandler), "파일 핸들러가 아님"
    
    # 둘 다 활성화
    logger_both = setup_logger("test_both", enable_console=True, enable_file=True, log_dir=str(test_log_dir))
    assert len(logger_both.handlers) == 2, "콘솔+파일 로거에 핸들러가 2개가 아님"
    
    add_test_result("콘솔/파일 핸들러 선택적 활성화", "PASS", "콘솔/파일 선택 활성화 성공")
except Exception as e:
    add_test_result("콘솔/파일 핸들러 선택적 활성화", "FAIL", str(e))

[PASS] 콘솔/파일 핸들러 선택적 활성화: 콘솔/파일 선택 활성화 성공


## Test 7: 로거 초기화

In [10]:
try:
    # 로거 생성
    logger1 = setup_logger("test_reset_1", log_dir=str(test_log_dir))
    logger2 = setup_logger("test_reset_2", log_dir=str(test_log_dir))
    
    # 초기 핸들러 수 저장
    handlers_count_1 = len(logger1.handlers)
    handlers_count_2 = len(logger2.handlers)
    
    # 특정 로거 초기화
    reset_logger("test_reset_1")
    
    # 핸들러가 제거되었는지 확인
    assert len(logger1.handlers) == 0, "reset_logger 후 핸들러가 제거되지 않음"
    
    # 새로 생성하면 핸들러가 다시 추가되어야 함
    logger1_after = setup_logger("test_reset_1", log_dir=str(test_log_dir))
    assert len(logger1_after.handlers) == handlers_count_1, "재생성 후 핸들러 수가 일치하지 않음"
    
    # 전체 로거 초기화
    reset_all_loggers()
    assert len(logger2.handlers) == 0, "reset_all_loggers 후 핸들러가 제거되지 않음"
    
    add_test_result("로거 초기화", "PASS", "reset_logger, reset_all_loggers 핸들러 초기화 확인")
except Exception as e:
    add_test_result("로거 초기화", "FAIL", str(e))

[PASS] 로거 초기화: reset_logger, reset_all_loggers 핸들러 초기화 확인


## 테스트 결과 요약

In [13]:
import pandas as pd

# 결과 요약
df_results = pd.DataFrame(test_results)
print("\n" + "="*80)
print("테스트 결과 요약")
print("="*80)
print(df_results.to_string(index=False))
print("="*80)

# 통계
total_tests = len(test_results)
passed_tests = sum(1 for r in test_results if r["상태"] == "PASS")
failed_tests = sum(1 for r in test_results if r["상태"] == "FAIL")

print(f"\n총 테스트: {total_tests}")
print(f"성공: {passed_tests}")
print(f"실패: {failed_tests}")
print(f"성공률: {passed_tests/total_tests*100:.1f}%")


테스트 결과 요약
                  테스트명   상태                                                                              상세내용
         로거 생성 및 기본 설정 PASS                                                                  로거 생성 및 로그 작성 성공
     파일 핸들러 및 로그 파일 생성 PASS 로그 파일: /home/spai0433/DATA/Dev/Work/codeit_ai_g2b_search/logs/test/rag_system.log
           로거 캐싱 및 재사용 PASS                                                                    동일 인스턴스 재사용 확인
              로깅 레벨 변경 PASS                                                                INFO → DEBUG 변경 성공
ShortLevelFormatter 동작 PASS                                                                   D/I/W/E/C 축약 확인
     콘솔/파일 핸들러 선택적 활성화 PASS                                                                   콘솔/파일 선택 활성화 성공
                로거 초기화 PASS                                        reset_logger, reset_all_loggers 핸들러 초기화 확인

총 테스트: 7
성공: 7
실패: 0
성공률: 100.0%


## 테스트 리포트 저장

In [14]:
# 리포트 파일 생성
report_dir = project_root / "scripts" / "test_report"
report_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
report_file = report_dir / f"test_log_{timestamp}.md"

# 마크다운 리포트 작성
report_content = f"""# logging_config 모듈 테스트 리포트

**테스트 일시**: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}  
**테스트 대상**: `src/utils/logging_config.py`  
**테스트 파일**: `scripts/test_log.ipynb`

---

## 테스트 결과 요약

| 항목 | 값 |
|------|-----|
| 총 테스트 수 | {total_tests} |
| 성공 | {passed_tests} |
| 실패 | {failed_tests} |
| 성공률 | {passed_tests/total_tests*100:.1f}% |

---

## 상세 테스트 결과

| # | 테스트명 | 상태 | 상세내용 |
|---|----------|------|----------|
"""

for idx, result in enumerate(test_results, 1):
    status_icon = "✅" if result["상태"] == "PASS" else "❌"
    report_content += f"| {idx} | {result['테스트명']} | {status_icon} {result['상태']} | {result['상세내용']} |\n"

report_content += f"""
---

## 테스트 항목 상세

### 1. 로거 생성 및 기본 설정
- 로거 인스턴스 생성 확인
- 핸들러 추가 확인
- 기본 로그 작성 테스트 (INFO, DEBUG, WARNING, ERROR)

### 2. 파일 핸들러 및 로그 파일 생성
- 로그 파일 생성 확인
- 로그 파일 경로 반환 확인
- 파일에 로그 내용 기록 확인

### 3. 로거 캐싱 및 재사용
- `setup_logger` 중복 호출 시 동일 인스턴스 반환 확인
- `get_logger` 호출 시 캐시된 로거 반환 확인

### 4. 로깅 레벨 변경
- `set_level` 함수로 레벨 변경 확인
- 핸들러 레벨도 함께 변경되는지 확인

### 5. ShortLevelFormatter 동작 확인
- DEBUG → D
- INFO → I
- WARNING → W
- ERROR → E
- CRITICAL → C

### 6. 콘솔/파일 핸들러 선택적 활성화
- `enable_console=True, enable_file=False`: 콘솔만
- `enable_console=False, enable_file=True`: 파일만
- `enable_console=True, enable_file=True`: 둘 다

### 7. 로거 초기화
- `reset_logger`: 특정 로거 초기화
- `reset_all_loggers`: 모든 로거 초기화

---

## 결론

{f"모든 테스트가 성공적으로 통과했습니다. ✅" if failed_tests == 0 else f"{failed_tests}개의 테스트가 실패했습니다. 상세 내용을 확인하세요. ❌"}

**테스트 로그 디렉토리**: `{test_log_dir}`  
**리포트 파일**: `{report_file}`
"""

# 파일 저장
report_file.write_text(report_content, encoding='utf-8')
print(f"\n테스트 리포트 저장 완료: {report_file}")


테스트 리포트 저장 완료: /home/spai0433/DATA/Dev/Work/codeit_ai_g2b_search/scripts/test_report/test_log_20251110_085524.md
